# Импорт

In [1]:
pip install geojson

In [ ]:
import pandas as pd
import numpy as np
import warnings
import folium
import geojson
from google.colab import drive
from joblib import dump, load
warnings.filterwarnings("ignore")
drive.mount('/final_project/')

%cd "/final_project/MyDrive/final_project"
%ls

# Функции

In [3]:
def get_geojson_features(regions, data):
  """
  Создаем geojson из заранее сформированных dataframe
  """
  feature_list = []
  for i in range(len(regions)):
    geometry = geojson.Polygon([[[regions.iloc[i][0], regions.iloc[i][2]],
                                [regions.iloc[i][0], regions.iloc[i][3]],
                                [regions.iloc[i][1], regions.iloc[i][3]],
                                [regions.iloc[i][1], regions.iloc[i][2]],
                                [regions.iloc[i][0], regions.iloc[i][2]]]])
    region_num = regions.iloc[i].name
    forecast1 = str(data[data.region_num == str(region_num)].forecast_1hour.values[0])
    forecast2 = str(data[data.region_num == str(region_num)].forecast_2hour.values[0])
    forecast3 = str(data[data.region_num == str(region_num)].forecast_3hour.values[0])
    forecast4 = str(data[data.region_num == str(region_num)].forecast_4hour.values[0])
    forecast5 = str(data[data.region_num == str(region_num)].forecast_5hour.values[0])
    forecast6 = str(data[data.region_num == str(region_num)].forecast_6hour.values[0])
    trips_last_hour = str(data[data.region_num == str(region_num)].count_trips.values[0])

    
    feature = geojson.Feature(id=str(i), geometry=geometry, properties={'region': str(region_num),
                                                                        'trips_last_hour': trips_last_hour, 
                                                                        'forecast_1hour': forecast1,
                                                                        'forecast_2hour': forecast2,
                                                                        'forecast_3hour': forecast3,
                                                                        'forecast_4hour': forecast4,
                                                                        'forecast_5hour': forecast5,
                                                                        'forecast_6hour': forecast6})
    feature_list.append(feature)

  return geojson.FeatureCollection(feature_list)

### Строим интерактивную карту в folium

In [4]:
NY_CENTER = [np.mean([40.49612, 40.91553]), np.mean([-74.25559, -73.70001])]

# загружаем таблицу с прогнозом на 1-6 часов вперед, которую сделали на прошлой неделе
LAST_FORECAST_DATAFRAME = load('last_forecast.joblib')

# загружаем таблицу с координатами регионов. Для отрисовки карты
regions_with_lat_lon = load('regions_coord.joblib')

# создаем geojson
coord = get_geojson_features(regions_with_lat_lon, LAST_FORECAST_DATAFRAME)

# создаем карту
m = folium.Map(location=NY_CENTER, width=600, height=600, zoom_start=11)
choropleth = folium.Choropleth(geo_data=coord,
                  data=LAST_FORECAST_DATAFRAME, columns=['region_num', 'count_trips'],
                  key_on='feature.properties.region',
                  bins=6,
                  fill_color='YlOrRd',
                  nan_fill_color='white', 
                  nan_fill_opacity=0.7,
                  fill_opacity=0.8, 
                  line_opacity=0.1,
                  legend_name='Количество заказов такси').add_to(m)

folium.LayerControl().add_to(m)
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['region', 'trips_last_hour', 'forecast_1hour', 'forecast_2hour', 
                                    'forecast_3hour','forecast_4hour','forecast_5hour','forecast_6hour'], 
                                   aliases=['Region number', 'Trips in last hour', 'Forecast for 1 hour ahead', 'Forecast for 2 hour ahead', 
                                            'Forecast for 3 hour ahead', 'Forecast for 4 hour ahead', 'Forecast for 5 hour ahead', 'Forecast for 6 hour ahead'], 
                                   labels=True, sticky=True)
                                   
)
m.save('interactive_map.html')
m